In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html # cuda version
!pip install dgl

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/Github version/')

In [4]:
import warnings
warnings.filterwarnings('ignore')
from model import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import scipy.sparse as sp
import networkx as nx
from scipy.sparse import coo_matrix
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device="cpu"

In [6]:
'''
converting to sparse matrix
'''

label_data=pd.read_csv('/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/Records/label_music.txt',header=None)
label=label_data[0].tolist()

LEN=len(label)
nl=coo_matrix((LEN, LEN))
nl.setdiag(1)
values = nl.data
indices = np.vstack((nl.row, nl.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = nl.shape
nl=torch.sparse_coo_tensor(i, v, torch.Size(shape))

In [7]:
nl=np.eye(LEN)
nl=torch.from_numpy(nl)

In [8]:
chemicalsub_drug = torch.load('/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/Records/hyG_music_kmer_25.pt')
data_dict = {
        ('node', 'in', 'edge'): (chemicalsub_drug[:,0], chemicalsub_drug[:,1]),        
        ('edge', 'con', 'node'): (chemicalsub_drug[:,1], chemicalsub_drug[:,0])
    }
'''
finally passing the data_dict to construct hyG
'''
hyG = dgl.heterograph(data_dict)
n_chemicalsub=18752  #change the num of rows based on the dataset. row info. is available on data folder
rows=n_chemicalsub
n_hedge=LEN
columns=n_hedge

'''
laoding the feature (one hot coding) for sequences (edges)
'''
sequence_X=nl

v_feat=coo_matrix((rows, 128))
v_feat.setdiag(1)
values = v_feat.data
indices = np.vstack((v_feat.row, v_feat.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = v_feat.shape
v_feat=torch.sparse_coo_tensor(i, v, torch.Size(shape))

hyG.ndata['h'] = {'edge' : sequence_X.type('torch.FloatTensor'), 'node' : v_feat.type('torch.FloatTensor')}
e_feat = sequence_X.type('torch.FloatTensor')
v_feat=v_feat.type('torch.FloatTensor')

In [9]:
hyG

Graph(num_nodes={'edge': 2221, 'node': 18752},
      num_edges={('edge', 'con', 'node'): 34111, ('node', 'in', 'edge'): 34111},
      metagraph=[('edge', 'node', 'con'), ('node', 'edge', 'in')])

In [10]:
label_data=pd.read_csv('/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/Records/label_music.txt',header=None)
label=label_data[0].tolist()
num_class=len(set(label))

test_size=0.1
from sklearn.model_selection import train_test_split
train_label, test_label= train_test_split( label,test_size=test_size, random_state=42)
train_label=torch.tensor(train_label,dtype=torch.long).to(device)
size=int((LEN)*0.1)
val_label=train_label[0:size]
train_label=train_label[size:]
test_label=torch.tensor(test_label,dtype=torch.long).to(device)

v_feat=v_feat.to(device)
e_feat=e_feat.to(device)
hyG=hyG.to(device)

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc

import torch
import torch.nn as nn
import numpy as np
from torch.nn import Parameter
import torch.optim as optim
from tqdm import tqdm
import pdb
import time

leanring_model =Seq_HyGAN(sequence_X.shape[1], 64, 128, 128,  num_class, 0.3)
leanring_model=leanring_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(leanring_model.parameters(), lr=0.01)
best_val_acc = 0
patience=0

for i in range(200):
    leanring_model.train()
    pred = leanring_model(hyG, v_feat, e_feat, True,True)
     
    train_pred, test_pred= train_test_split( pred,test_size=test_size, random_state=42)
    val_pred=train_pred[0:size]
    train_pred=train_pred[size:]

    loss = loss_fn(train_pred, train_label)
    pred_cls = torch.argmax(train_pred, -1)
    train_acc = torch.eq(pred_cls, train_label).sum().item()/len(train_label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
      leanring_model.eval()
      val_cls = torch.argmax(val_pred, -1) 
      val_acc = torch.eq(val_cls, val_label).sum().item()/len(val_label)

      # Save the best validation accuracy and the corresponding test accuracy.
      if best_val_acc < val_acc:
        best_val_acc = val_acc
        E=i
        patience=0
        torch.save(test_pred,'latest.pth')
      else:
        patience+=1

    if patience==200:
      break

    if i % 10 == 0:
      print('In epoch {}, train loss: {:.4f}, val_acc: {:.4f} (best_val_acc: {:.4f})'.format(i, loss, val_acc, best_val_acc))

In [ ]:
best_test_perd=torch.load('latest.pth')
with torch.no_grad():
  test_pred=best_test_perd
  test_cls = torch.argmax(test_pred, -1) 
  test_acc = torch.eq(test_cls, test_label).sum().item()/len(test_label)
  print(' Best Epoch: {}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score {:.4f}'.format(E,accuracy_score(test_label.cpu(),test_cls.cpu()),precision_score(test_label.cpu(),test_cls.cpu(),average='weighted'),recall_score(test_label.cpu(),test_cls.cpu(),average='weighted'),f1_score(test_label.cpu(),test_cls.cpu(),average='weighted')))